### Jupyter Notebook: Verification Suite for Entry/Exit Handlers

#### Cell 1: Setup - Function Definitions and Imports

This cell remains the same, defining the functions we need to test.

In [1]:
import pandas as pd

def handle_entries_for_day(current_date, next_day_date, signals_today, open_positions, df_ohlcv):
    """
    Processes entries and stores signal details in the open_positions dict.
    """
    # --- KEY CHANGE: Loop through the signals DataFrame ---
    for ticker, signal_row in signals_today.iterrows():
        # The ticker is now in the index of signal_row, so we use its name
        ticker_name = ticker[0] 
        
        if ticker_name not in open_positions:
            try:
                entry_price = df_ohlcv.loc[(ticker_name, next_day_date), 'Adj High']
                
                # --- LOGGING: Store more info about the entry signal ---
                open_positions[ticker_name] = {
                    'entry_date': next_day_date,
                    'entry_price': entry_price,
                    'signal_date': current_date,
                    'signal_features': signal_row.to_dict() # Store all features that triggered the signal
                }
            except KeyError:
                pass
                
    return open_positions

def handle_exits_for_day(current_date, next_day_date, open_positions, df_ohlcv, config):
    """
    Checks for exits and logs detailed information about the exit trigger.
    Corrected version with valid syntax for the if/elif chain.
    """
    closed_trades = []
    positions_to_close = []

    for ticker, pos in open_positions.items():
        try:
            current_close_price = df_ohlcv.loc[(ticker, current_date), 'Adj Close']
        except KeyError:
            continue 

        exit_reason = None
        exit_target_value = None 
        
        # --- SYNTAX FIX: Calculate all threshold values *before* the conditional block ---
        profit_target_price = pos['entry_price'] * (1 + config['profit_target'])
        stop_loss_price = pos['entry_price'] * (1 - config['stop_loss'])
        days_held = (current_date.to_pydatetime().date() - pos['entry_date'].to_pydatetime().date()).days
        
        # --- Now, check conditions in a contiguous if/elif/elif block ---
        if current_close_price >= profit_target_price:
            exit_reason = "Profit Target"
            exit_target_value = profit_target_price 

        elif current_close_price <= stop_loss_price:
            exit_reason = "Stop-Loss"
            exit_target_value = stop_loss_price 

        elif days_held >= config['time_hold_days']:
            exit_reason = "Time Hold"
            exit_target_value = days_held 

        if exit_reason:
            try:
                exit_price = df_ohlcv.loc[(ticker, next_day_date), 'Adj Low']
                trade_return = (exit_price - pos['entry_price']) / pos['entry_price']
                
                trade_log = {
                    'ticker': ticker, 
                    'entry_date': pos['entry_date'], 
                    'exit_date': next_day_date,
                    'return': trade_return, 
                    'reason': exit_reason,
                    'signal_date': pos['signal_date'],
                    'entry_signal_features': pos['signal_features'],
                    'entry_price_actual': pos['entry_price'],
                    'exit_signal_date': current_date,
                    'exit_trigger_price': current_close_price,
                    'exit_target_value': exit_target_value,
                    'exit_price_actual': exit_price,
                }
                closed_trades.append(trade_log)
                positions_to_close.append(ticker)
            except KeyError:
                pass
                
    for ticker in positions_to_close:
        del open_positions[ticker]
        
    return closed_trades, open_positions



print("Setup complete. Entry and Exit handlers are defined.")

Setup complete. Entry and Exit handlers are defined.


#### Cell 2: Test Data and Configuration

This cell defines our shared "universe" of data and the configuration that will be used across all test scenarios.

In [2]:
# --- Shared Test Configuration ---
# This dictionary contains the strategy parameters for our tests.
test_config = {
    'profit_target': 0.10, # Exit if price rises 10% above entry
    'stop_loss': 0.05,     # Exit if price drops 5% below entry
    'time_hold_days': 2     # Exit if held for 2 or more days
}

# --- Shared Test Data ---
# This handcrafted DataFrame creates a predictable "story" for three tickers.
# - TICKA is designed to hit a Stop-Loss.
# - TICKB is designed to hit a Profit Target.
# - TICKC is designed to hit a Time Hold exit.
data = {
    'Ticker': ['TICKA', 'TICKA', 'TICKA', 'TICKA', 'TICKA', 
               'TICKB', 'TICKB', 'TICKB', 'TICKB', 'TICKB', 
               'TICKC', 'TICKC', 'TICKC', 'TICKC', 'TICKC'],
    'Date': pd.to_datetime(['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',               
                           '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
                           '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']),
    # Entry prices (T+1 High): TICKA=101, TICKB=51, TICKC=50
    'Adj High':  [100, 101, 102, 98,  95,   50,  51,  58,  59,  55,   50,  50,  50,  50,  50],
    # Exit prices (T+1 Low): TICKA=93 (Day 5), TICKB=57 (Day 4), TICKC=48 (Day 4)
    'Adj Low':   [98,  99,  100, 94,  93,   49,  50,  56,  57,  52,   48,  48,  48,  48,  48],
    # Trigger prices (T Close): TICKA=95 (Day 4), TICKB=57 (Day 3), TICKC=49 (Day 3)
    'Adj Close': [99,  100, 101, 95,  94,   50,  50,  57,  58,  53,   49,  49,  49,  49,  49],
}
test_df_ohlcv = pd.DataFrame(data).set_index(['Ticker', 'Date'])

print("--- Test Data and Config ---")
print("Config:", test_config)
print("\nOHLCV DataFrame:")
print(test_df_ohlcv)

--- Test Data and Config ---
Config: {'profit_target': 0.1, 'stop_loss': 0.05, 'time_hold_days': 2}

OHLCV DataFrame:
                   Adj High  Adj Low  Adj Close
Ticker Date                                    
TICKA  2023-01-02       100       98         99
       2023-01-03       101       99        100
       2023-01-04       102      100        101
       2023-01-05        98       94         95
       2023-01-06        95       93         94
TICKB  2023-01-02        50       49         50
       2023-01-03        51       50         50
       2023-01-04        58       56         57
       2023-01-05        59       57         58
       2023-01-06        55       52         53
TICKC  2023-01-02        50       48         49
       2023-01-03        50       48         49
       2023-01-04        50       48         49
       2023-01-05        50       48         49
       2023-01-06        50       48         49


#### Cell 3: Scenario 1 - Verification of Stop-Loss Exit

This test focuses *only* on the story of TICKA.

In [3]:
print("--- SCENARIO 1: Verifying a Stop-Loss Exit ---")

# --- ARRANGE: Initial State ---
# We start with no open positions and no trades.
open_positions = {}
trades = []
all_dates = test_df_ohlcv.index.get_level_values('Date').unique()
current_date, next_day_date = all_dates[0], all_dates[1] # Day 1 -> Day 2

# --- ACT 1: Entry ---
# Simulate a signal for TICKA on Day 1, for entry on Day 2.
print(f"\n1. ENTRY: Simulating signal for TICKA on {current_date.date()} to enter on {next_day_date.date()}")
signal_index = pd.MultiIndex.from_tuples([('TICKA', current_date)], names=['Ticker', 'Date'])
signals_today = pd.DataFrame(index=signal_index)
open_positions = handle_entries_for_day(current_date, next_day_date, signals_today, open_positions, test_df_ohlcv)

# --- ASSERT 1: Verify Entry ---
print(f"   Open positions after entry: {open_positions}")
assert 'TICKA' in open_positions, "FAIL: TICKA was not opened."
assert open_positions['TICKA']['entry_price'] == 101.0, "FAIL: TICKA entry price is wrong."
print("   [SUCCESS] TICKA correctly entered a position.")

# --- ACT 2: Hold ---
# Simulate holding through Day 2 and Day 3. Nothing should be closed.
current_date, next_day_date = all_dates[1], all_dates[2] # Day 2
closed_trades, open_positions = handle_exits_for_day(current_date, next_day_date, open_positions, test_df_ohlcv, test_config)
assert len(closed_trades) == 0, "FAIL: Trade closed prematurely on Day 2."

current_date, next_day_date = all_dates[2], all_dates[3] # Day 3
closed_trades, open_positions = handle_exits_for_day(current_date, next_day_date, open_positions, test_df_ohlcv, test_config)
assert len(closed_trades) == 0, "FAIL: Trade closed prematurely on Day 3."
print("\n2. HOLD: Position correctly held for 2 days.")
print("   [SUCCESS] No premature exits occurred.")

# --- ACT 3: Exit ---
# On Day 4, the close price (95) should trigger the stop-loss (101 * 0.95 = 95.95).
current_date, next_day_date = all_dates[3], all_dates[4] # Day 4 -> Day 5
print(f"\n3. EXIT: Simulating stop-loss signal on {current_date.date()} to exit on {next_day_date.date()}")
closed_trades, open_positions = handle_exits_for_day(current_date, next_day_date, open_positions, test_df_ohlcv, test_config)
trades.extend(closed_trades)

# --- ASSERT 3: Verify Exit ---
print(f"   Closed trades found: {len(trades)}")
print(f"   Remaining open positions: {open_positions}")
assert 'TICKA' not in open_positions, "FAIL: TICKA was not removed from open positions."
assert len(trades) == 1, "FAIL: The trade was not logged."
trade_log = trades[0]
assert trade_log['reason'] == 'Stop-Loss', "FAIL: Exit reason is incorrect."
assert trade_log['exit_price_actual'] == 93.0, "FAIL: Exit price is incorrect (should be Day 5 Low)."
print("   [SUCCESS] TICKA correctly exited with Stop-Loss.")
print("\n--- SCENARIO 1 COMPLETE ---")

--- SCENARIO 1: Verifying a Stop-Loss Exit ---

1. ENTRY: Simulating signal for TICKA on 2023-01-02 to enter on 2023-01-03
   Open positions after entry: {'TICKA': {'entry_date': Timestamp('2023-01-03 00:00:00'), 'entry_price': 101, 'signal_date': Timestamp('2023-01-02 00:00:00'), 'signal_features': {}}}
   [SUCCESS] TICKA correctly entered a position.

2. HOLD: Position correctly held for 2 days.
   [SUCCESS] No premature exits occurred.

3. EXIT: Simulating stop-loss signal on 2023-01-05 to exit on 2023-01-06
   Closed trades found: 1
   Remaining open positions: {}
   [SUCCESS] TICKA correctly exited with Stop-Loss.

--- SCENARIO 1 COMPLETE ---


#### Cell 4: Scenario 2 - Verification of Profit Target Exit

This test focuses *only* on the story of TICKB.

In [ ]:
print("\n--- SCENARIO 2: Verifying a Profit Target Exit ---")

# --- ARRANGE: Initial State ---
# Simulate that TICKB was entered on Day 2 at a price of 51.0
open_positions = {'TICKB': {'entry_date': pd.to_datetime('2023-01-03'), 'entry_price': 51.0, 
                            'signal_date': pd.to_datetime('2023-01-02'), 'signal_features': {}}}
trades = []
all_dates = test_df_ohlcv.index.get_level_values('Date').unique()

# --- ACT: Exit ---
# On Day 3, the close price (57) should trigger the profit target (51 * 1.10 = 56.1).
current_date, next_day_date = all_dates[2], all_dates[3] # Day 3 -> Day 4
print(f"\n1. EXIT: Simulating profit target signal on {current_date.date()} to exit on {next_day_date.date()}")
closed_trades, open_positions = handle_exits_for_day(current_date, next_day_date, open_positions, test_df_ohlcv, test_config)
trades.extend(closed_trades)

# --- ASSERT: Verify Exit ---
print(f"   Closed trades found: {len(trades)}")
print(f"   Remaining open positions: {open_positions}")
assert 'TICKB' not in open_positions, "FAIL: TICKB was not removed from open positions."
assert len(trades) == 1, "FAIL: The trade was not logged."
trade_log = trades[0]
assert trade_log['reason'] == 'Profit Target', "FAIL: Exit reason is incorrect."
assert trade_log['exit_price_actual'] == 57.0, "FAIL: Exit price is incorrect (should be Day 4 Low)."
print("   [SUCCESS] TICKB correctly exited with Profit Target.")
print("\n--- SCENARIO 2 COMPLETE ---")


--- SCENARIO 2: Verifying a Profit Target Exit ---

1. EXIT: Simulating profit target signal on 2023-01-04 to exit on 2023-01-05
   Closed trades found: 1
   Remaining open positions: {}
   [SUCCESS] TICKB correctly exited with Profit Target.

--- SCENARIO 2 COMPLETE ---


#### Cell 5: Scenario 3 - Verification of Time Hold Exit

This test focuses *only* on the story of TICKC.

In [ ]:
print("\n--- SCENARIO 3: Verifying a Time Hold Exit ---")

# --- ARRANGE: Initial State ---
# Simulate that TICKC was entered on Day 2 at a price of 50.0
open_positions = {'TICKC': {'entry_date': pd.to_datetime('2023-01-03'), 'entry_price': 50.0,
                            'signal_date': pd.to_datetime('2023-01-02'), 'signal_features': {}}}
trades = []
all_dates = test_df_ohlcv.index.get_level_values('Date').unique()

# --- ACT: Exit ---
# On Day 3, the days_held is (Jan 4 - Jan 3) = 1 day. No exit.
current_date, next_day_date = all_dates[2], all_dates[3]
closed_trades, open_positions = handle_exits_for_day(current_date, next_day_date, open_positions, test_df_ohlcv, test_config)
assert len(closed_trades) == 0, "FAIL: Trade closed prematurely on Day 3."
print(f"\n1. HOLD: Position correctly held on {current_date.date()} (Days held = 1).")


# On Day 4, the days_held is (Jan 5 - Jan 3) = 2 days. This should trigger the exit (time_hold_days >= 2).
current_date, next_day_date = all_dates[3], all_dates[4] # Day 4 -> Day 5
print(f"\n2. EXIT: Simulating time hold signal on {current_date.date()} to exit on {next_day_date.date()} (Days held = 2).")
closed_trades, open_positions = handle_exits_for_day(current_date, next_day_date, open_positions, test_df_ohlcv, test_config)
trades.extend(closed_trades)

# --- ASSERT: Verify Exit ---
print(f"   Closed trades found: {len(trades)}")
print(f"   Remaining open positions: {open_positions}")
assert 'TICKC' not in open_positions, "FAIL: TICKC was not removed from open positions."
assert len(trades) == 1, "FAIL: The trade was not logged."
trade_log = trades[0]
assert trade_log['reason'] == 'Time Hold', "FAIL: Exit reason is incorrect."
assert trade_log['exit_price_actual'] == 48.0, "FAIL: Exit price is incorrect (should be Day 5 Low)."
assert trade_log['exit_target_value'] == 2, "FAIL: Logged days held is incorrect."
print("   [SUCCESS] TICKC correctly exited with Time Hold.")
print("\n--- SCENARIO 3 COMPLETE ---")


--- SCENARIO 3: Verifying a Time Hold Exit ---

1. HOLD: Position correctly held on 2023-01-04 (Days held = 1).

2. EXIT: Simulating time hold signal on 2023-01-05 to exit on 2023-01-06 (Days held = 2).
   Closed trades found: 1
   Remaining open positions: {}
   [SUCCESS] TICKC correctly exited with Time Hold.

--- SCENARIO 3 COMPLETE ---


### This structured approach provides a comprehensive and clean test suite. Each cell tests one specific piece of logic, making it very easy to read, maintain, and trust in the future.